# Real-world network

In [1]:
import networkx as nx
from networkx.algorithms.centrality import *
import numpy as np
import operator
from analysis import *

In [2]:
g = nx.read_gml("./RealNetworks/hamsterster.gml")

In [3]:
print(nx.info(g))

Graph with 1788 nodes and 12476 edges


<ipython-input-3-99dac2172e06>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))


# Check connectedness

In [4]:
nx.is_connected(g)

True

In [5]:
nx.number_connected_components(g)

1

# Get LCC if graph is disconnected

# Relabel it all if nodes are strings
- To be able to run all types of correlation together
- But put the graph relabled in `g_relabled`

In [6]:
from myTruss import mappingAndRelabeling

In [7]:
g_relabled = mappingAndRelabeling(g)

In [8]:
print(nx.info(g))
print(nx.info(g_relabled))

Graph with 1788 nodes and 12476 edges
Graph with 1788 nodes and 12476 edges


<ipython-input-8-632c4a2d19f6>:1: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g))
<ipython-input-8-632c4a2d19f6>:2: DeprecationWarning: info is deprecated and will be removed in version 3.0.

  print(nx.info(g_relabled))


In [9]:
g = g_relabled

# Infer community structure with Louvain

In [10]:
nx.__version__

'2.8.4'

In [11]:
import pandas as pd
from os.path  import exists
import infomap as im

In [12]:
im.__version__

'2.3.0'

In [13]:
G = g

In [14]:
partition   = nx.algorithms.community.louvain_communities(G, seed = 0)
partition_Q = nx.algorithms.community.modularity(G, partition)

for seed in Bar("Progress", check_tty = False).iter(range(1,5)):
    partition_   = nx.algorithms.community.louvain_communities(G, seed = seed)
    partition_Q_ = nx.algorithms.community.modularity(G, partition_)
    
    if partition_Q_ > partition_Q:
        partition   = partition_
        partition_Q = partition_Q_

Progress |################################| 4/4


In [15]:
communities = { community_ID : set(community) for (community_ID, community) in enumerate(partition, start = 1)}

In [16]:
community_assignments = dict()
for (community_ID, community) in communities.items():
    for node in community:
        community_assignments[node] = community_ID

In [17]:
results_file = "./results/hamsterster-modularity.csv"

if not exists(results_file):
    # Map Equation Centrality
    tree_file = "./results/hamsterster-temp.tree"

    # use Infomap to write a tree file that we change to reflect the Louvain partition
    infomap = im.Infomap(silent = True, num_trials = 1, directed = False, two_level = True, seed = 42)
    infomap.add_networkx_graph(G)
    infomap.run()
    infomap.write_tree(tree_file)
    
    infomap_tree = dict()

    # read the tree file
    with open(tree_file, "r") as fh:
        for line in fh:
            if not line.startswith("#"):
                _, flow, name, node_id = line.strip().split(" ")
                name = name[1:-1]
                infomap_tree[int(name)] = (flow, node_id)
    
    # write the Louvain partition to an Infomap tree file
    with open("./results/hamsterster-modularity.tree", "w") as fh:
        for community_ID, community in enumerate(partition, start = 1):
            for node_pos, node in enumerate(community, start = 1):
                path = f"{community_ID}:{node_pos}"
                flow, node_id = infomap_tree[node]
                fh.write(f"{path} {flow} \"{node}\" {node_id}\n")
    
    # Load the Louvain partition with Infomap and use unrecorded link teleportation
    infomap2 = im.Infomap("--silent --no-infomap --cluster-data ./results/hamsterster-modularity.tree")
    node_mapping = infomap2.add_networkx_graph(G)
    infomap2.run()
    
    mec_ranks = [node for node, _score in sorted( [ (node_mapping[node.node_id], node.modular_centrality) for node in infomap2.iterLeafNodes() ]
                                                 , key     = second
                                                 , reverse = True
                                                 )]

    
    index = []
    for node in G.nodes:
        index.append(node)

    results = pd.DataFrame(index=index)

    results["map equation centrality rank"]       = toRanking(mec_ranks,        results.index)
    results.to_csv(results_file)

else:
    results = pd.read_csv(results_file, index_col = 0)

    mec_ranks         = mkRanks(results["map equation centrality rank"])

In [18]:
len(mec_ranks)

1788

In [19]:
# node 107 has rank 0 (i.e. 1)
# node 1684 has rank 1 (i.e. 2)
# ...
# node 0 has rank 4 (i.e. 5)
mec_ranks

[91,
 90,
 31,
 93,
 15,
 132,
 30,
 29,
 59,
 58,
 28,
 56,
 84,
 57,
 100,
 24,
 51,
 39,
 161,
 8,
 159,
 147,
 448,
 87,
 109,
 81,
 201,
 246,
 825,
 83,
 50,
 250,
 89,
 230,
 18,
 148,
 92,
 259,
 331,
 149,
 225,
 78,
 118,
 409,
 52,
 134,
 871,
 126,
 672,
 85,
 133,
 80,
 493,
 77,
 79,
 14,
 205,
 203,
 106,
 390,
 9,
 350,
 76,
 143,
 323,
 403,
 94,
 260,
 267,
 172,
 204,
 62,
 86,
 171,
 671,
 168,
 379,
 45,
 88,
 239,
 668,
 266,
 108,
 354,
 273,
 353,
 822,
 268,
 674,
 105,
 634,
 727,
 35,
 322,
 101,
 616,
 60,
 129,
 130,
 673,
 389,
 238,
 302,
 468,
 309,
 127,
 36,
 584,
 270,
 257,
 33,
 128,
 136,
 196,
 256,
 324,
 190,
 75,
 356,
 278,
 1,
 632,
 726,
 99,
 522,
 326,
 481,
 343,
 247,
 124,
 70,
 269,
 114,
 494,
 40,
 594,
 414,
 853,
 74,
 692,
 406,
 97,
 98,
 255,
 725,
 254,
 252,
 334,
 463,
 358,
 72,
 112,
 264,
 560,
 610,
 244,
 73,
 737,
 724,
 599,
 123,
 351,
 107,
 5,
 864,
 253,
 82,
 501,
 160,
 304,
 96,
 551,
 1134,
 332,
 344,
 32,
 27

In [20]:
counter = len(mec_ranks)
dict_map_eq_centrality = {}

In [21]:
for node in mec_ranks:
    dict_map_eq_centrality[int(node)] = counter
    counter = counter - 1

In [22]:
dict_map_eq_centrality

{91: 1788,
 90: 1787,
 31: 1786,
 93: 1785,
 15: 1784,
 132: 1783,
 30: 1782,
 29: 1781,
 59: 1780,
 58: 1779,
 28: 1778,
 56: 1777,
 84: 1776,
 57: 1775,
 100: 1774,
 24: 1773,
 51: 1772,
 39: 1771,
 161: 1770,
 8: 1769,
 159: 1768,
 147: 1767,
 448: 1766,
 87: 1765,
 109: 1764,
 81: 1763,
 201: 1762,
 246: 1761,
 825: 1760,
 83: 1759,
 50: 1758,
 250: 1757,
 89: 1756,
 230: 1755,
 18: 1754,
 148: 1753,
 92: 1752,
 259: 1751,
 331: 1750,
 149: 1749,
 225: 1748,
 78: 1747,
 118: 1746,
 409: 1745,
 52: 1744,
 134: 1743,
 871: 1742,
 126: 1741,
 672: 1740,
 85: 1739,
 133: 1738,
 80: 1737,
 493: 1736,
 77: 1735,
 79: 1734,
 14: 1733,
 205: 1732,
 203: 1731,
 106: 1730,
 390: 1729,
 9: 1728,
 350: 1727,
 76: 1726,
 143: 1725,
 323: 1724,
 403: 1723,
 94: 1722,
 260: 1721,
 267: 1720,
 172: 1719,
 204: 1718,
 62: 1717,
 86: 1716,
 171: 1715,
 671: 1714,
 168: 1713,
 379: 1712,
 45: 1711,
 88: 1710,
 239: 1709,
 668: 1708,
 266: 1707,
 108: 1706,
 354: 1705,
 273: 1704,
 353: 1703,
 822: 17

In [23]:
# Write Centrality
with open('./MapEquationCentralityResults/OSN-Hamsterster/dict_map_eq_centrality.txt', 'w') as f:
    for key, value in dict_map_eq_centrality.items():
        f.write('%s:%s\n' % (key, value))

In [24]:
# Read Centrality
dict_map_eq_centrality_read = dict()
with open('./MapEquationCentralityResults/OSN-Hamsterster/dict_map_eq_centrality.txt') as raw_data:
    for item in raw_data:
        if ':' in item:
            key,value = item.split(':', 1)
            value = value.replace('\n', '')
            dict_map_eq_centrality_read[int(key)]=float(value)
        else:
            pass # deal with bad lines of text here

In [25]:
dict_map_eq_centrality_read == dict_map_eq_centrality

True